#### Import Dependencies

In [1]:
# System tools
import os
import pickle
from dotenv import load_dotenv

# The 'Brain' and 'Math'
from langchain_groq import ChatGroq
from langchain_huggingface import HuggingFaceEmbeddings

# Data Loading & Vector Storage (from community & specialized packages)
from langchain_community.document_loaders import UnstructuredURLLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Building chains with raw LCEL primitives
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

#### Setup Models (Initialize the AI and Keys)

In [2]:
# Load API keys
os.chdir(r"C:\Users\shedd\Desktop\Personal Projects\gen_ai")
load_dotenv()

# Initialize Llama 3 on Groq and local embeddings
# Note: The underscore import 'langchain_groq', not 'langchain.chat_models'
llm = ChatGroq(model_name="llama-3.3-70b-versatile", temperature=0.5)
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [3]:
print(f"Groq API Key Detected: {'Yes' if os.environ.get('GROQ_API_KEY') else 'No'}")
print(f"Embedding Model Ready: {embeddings.model_name}")

Groq API Key Detected: Yes
Embedding Model Ready: all-MiniLM-L6-v2


#### Load & Split Data

In [4]:
urls = [
    "https://www.moneycontrol.com/news/business/markets/wall-street-rises-as-tesla-soars-on-ai-optimism-11351111.html",
    "https://www.moneycontrol.com/news/business/tata-motors-launches-punch-icng-price-starts-at-rs-7-1-lakh-11098751.html"
]

loader = UnstructuredURLLoader(urls=urls)
data = loader.load()

# Using the specialized 'langchain_text_splitters' package
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(data)

In [5]:
print(f"Total Articles Loaded: {len(data)}")
print(f"Total Chunks Created: {len(docs)}\n")

# Preview the first chunk to check for clean text
print("--- First Chunk Preview ---")
print(docs[0].page_content[:500])  # Show first 500 characters
print("\n--- Source Metadata ---")
print(docs[0].metadata)

Total Articles Loaded: 2
Total Chunks Created: 17

--- First Chunk Preview ---
English

Hindi

Gujarati

Specials

Hello, Login

Hello, Login

Log-inor Sign-Up

My Account

My Profile

My Portfolio

My Watchlist

My Alerts

My Messages

Price Alerts

My Profile

My PRO

My Portfolio

My Watchlist

My Alerts

My Messages

Price Alerts

Logout

Loans up to ₹50 LAKHS

Fixed Deposits

Credit CardsLifetime Free

Credit Score

Chat with Us

Download App

Follow us on:

Network 18

Go Ad-Free

My Alerts

>->MC_ENG_DESKTOP/MC_ENG_NEWS/MC_ENG_MARKETS_AS/MC_ENG_ROS_NWS_MKTS_AS_ATF_7

--- Source Metadata ---
{'source': 'https://www.moneycontrol.com/news/business/markets/wall-street-rises-as-tesla-soars-on-ai-optimism-11351111.html'}


#### Build Vector Index

In [10]:
# Convert all those text chunks into vector numbers and store them in FAISS
# This is where the "learning" actually happens
vector_index = FAISS.from_documents(docs, embeddings)

# We'll save this as a simple pickle file right here in the project folder
file_path = "vector_index.pkl"

with open(file_path, "wb") as f:
    pickle.dump(vector_index, f)

In [11]:
# check if the file was actually created
if os.path.exists(file_path):
    print(f"Success! Index saved to: {file_path}")
    
    # Quick sanity check: Ask the raw index a question without the AI model
    # We want to make sure it can actually find relevant chunks
    test_query = "price of Tiago iCNG"
    results = vector_index.similarity_search(test_query, k=2)
    
    print("\n--- Raw Search Results ---")
    for i, res in enumerate(results):
        print(f"Result {i+1}: {res.page_content[:150]}...") # Show first 150 chars
        print(f"Source: {res.metadata['source']}\n")
else:
    print("Error: The file wasn't saved. Check your permissions.")

Success! Index saved to: vector_index.pkl

--- Raw Search Results ---
Result 1: The company also said it has also introduced the twin-cylinder technology on its Tiago and Tigor models.

The Tiago iCNG is priced between Rs 6.55 lak...
Source: https://www.moneycontrol.com/news/business/tata-motors-launches-punch-icng-price-starts-at-rs-7-1-lakh-11098751.html

Result 2: Trending Topics

Sensex Live

Union Budget 2026

IPO This Week

Kuku IPO

Gold Rate Today

Tata Motors launches Punch iCNG, price starts at Rs 7.1 lak...
Source: https://www.moneycontrol.com/news/business/tata-motors-launches-punch-icng-price-starts-at-rs-7-1-lakh-11098751.html



#### The LCEL Pipeline (Ask Questions and Get Answers)

In [12]:
# Load the "Brain" back into memory
# We open the pickle file we saved earlier. This is crucial because in a real app,
# you don't want to rebuild the index every time a user asks a question.
with open("vector_index.pkl", "rb") as f:
    loaded_index = pickle.load(f)

# create the "Search Engine"
# We convert our static index into a 'retriever' that can actively search.
# I'm setting 'k=2' here, which tells it: "Don't read the whole internet, 
# just bring me the top 2 most relevant snippets." This saves time and tokens.
retriever = loaded_index.as_retriever(search_kwargs={"k": 2})

# Give the AI its instructions (The Prompt)
# This is where we control the personality and accuracy. 
# By saying "based ONLY on the following context," we prevent it from making things up (hallucinating).
template = """
You are a helpful financial assistant. 
Answer the question based ONLY on the following context.
If you don't know, just say "I don't know".

Context: 
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

# A helper to glue the text together
# The retriever returns a list of separate documents, but the LLM just wants one big string of text.
# This function joins them all together with newlines.
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Connect the dots (The Chain)
# This uses LCEL (LangChain Expression Language) to build a pipeline.
# Think of the '|' symbol as a pipe passing data to the next step:
#   1. Find relevant data (retriever) & Keep the user's question
#   2. Feed both into the Prompt
#   3. Send Prompt to the LLM (Groq)
#   4. Clean up the output so it's just a string
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [13]:
# run the whole pipeline with a real question
question = "what is the price of Tiago iCNG?"
final_answer = rag_chain.invoke(question)

print(f"Question: {question}")
print(f"Answer: {final_answer}")

Question: what is the price of Tiago iCNG?
Answer: The Tiago iCNG is priced between Rs 6.55 lakh and Rs 8.1 lakh.
